## 26250 - ST115 Project Code

Introduction: This project aims to find out whether LSE departments should use Twitter as a means to communicate with LSE students. The way we can explore the statement above is to carry out an analysis similar to social media analytics. A couple of factors that we can look at are the follower counts, total likes, retweets, quotes, mention sentiments, and engagement ratios.

### Part 1: Data Collection

First, we will call the list of departments at the LSE so that we know which accounts to look for on Twitter.

In [1]:
from bs4 import BeautifulSoup
import requests
import json

In [2]:
url = "https://info.lse.ac.uk/staff/departments-and-institutes"
response = requests.get(url)
soup = BeautifulSoup(response.content, "lxml")

In [3]:
raw_list = soup.find_all('ul')

In [4]:
dept = []

for tag in raw_list:
    list_el = tag.text.strip()
    dept.append(list_el)

In [5]:
depts = dept[2:]
depts

['Department of Accounting\nDepartment of Anthropology\nData Science Institute\nDepartment of Economics\nDepartment of Economic History\nEuropean Institute\nDepartment of Finance\nFiroz Lalji Institute for Africa\nDepartment of Gender Studies\nDepartment of Geography and Environment\nDepartment of Government\nDepartment of Health Policy\nDepartment of International Development\nDepartment of International History\nInternational Inequalities Institute\nDepartment of International Relations\nLanguage Centre\nLSE Law School\nDepartment of Management\nMarshall Institute\nDepartment of Mathematics\nDepartment of Media and Communications\nDepartment of Methodology\nDepartment of Philosophy, Logic and Scientific Method\nDepartment of Psychological and Behavioural Science\nSchool of Public Policy (formerly Institute of Public Affairs)\nDepartment of Social Policy\nDepartment of Sociology\nDepartment of Statistics']

In [6]:
dept_list = depts[0].split("\n")
dept_list

['Department of Accounting',
 'Department of Anthropology',
 'Data Science Institute',
 'Department of Economics',
 'Department of Economic History',
 'European Institute',
 'Department of Finance',
 'Firoz Lalji Institute for Africa',
 'Department of Gender Studies',
 'Department of Geography and Environment',
 'Department of Government',
 'Department of Health Policy',
 'Department of International Development',
 'Department of International History',
 'International Inequalities Institute',
 'Department of International Relations',
 'Language Centre',
 'LSE Law School',
 'Department of Management',
 'Marshall Institute',
 'Department of Mathematics',
 'Department of Media and Communications',
 'Department of Methodology',
 'Department of Philosophy, Logic and Scientific Method',
 'Department of Psychological and Behavioural Science',
 'School of Public Policy (formerly Institute of Public Affairs)',
 'Department of Social Policy',
 'Department of Sociology',
 'Department of Statisti

In [7]:
#Remove all elements in the list with the word Institute as we are looking only at Departments
indices = [dept_list.index("Data Science Institute"), dept_list.index("European Institute"), 
           dept_list.index("Firoz Lalji Institute for Africa"), dept_list.index("International Inequalities Institute"),
           dept_list.index("Marshall Institute"), 
           dept_list.index("School of Public Policy (formerly Institute of Public Affairs)")]

In [8]:
dept_list = [i for j, i in enumerate(dept_list) if j not in indices]
dept_list

['Department of Accounting',
 'Department of Anthropology',
 'Department of Economics',
 'Department of Economic History',
 'Department of Finance',
 'Department of Gender Studies',
 'Department of Geography and Environment',
 'Department of Government',
 'Department of Health Policy',
 'Department of International Development',
 'Department of International History',
 'Department of International Relations',
 'Language Centre',
 'LSE Law School',
 'Department of Management',
 'Department of Mathematics',
 'Department of Media and Communications',
 'Department of Methodology',
 'Department of Philosophy, Logic and Scientific Method',
 'Department of Psychological and Behavioural Science',
 'Department of Social Policy',
 'Department of Sociology',
 'Department of Statistics']

Based on the department list above, I've collected the list of Twitter usernames for each department manually because there isn't a compiled list of the department's Twitter handles.

In [9]:
usernames = ['LSE_Accounting', 'LSEAnthropology', 'LSEEcon',
            'LSEEcHist', 'LSEfinance', 'LSEGenderTweet', 'LSEGeography',
            'LSEGovernment', 'LSEHealthPolicy', 'LSE_ID', 'lsehistory',
            'LSEIRDept', 'lselangcentre', 'LSELaw', 'LSEManagement', 'LSEMaths',
            'MediaLSE', 'MethodologyLSE', 'LSEPhilosophy', 'LSEBehavioural', 
            'LSESocialPolicy', 'LSEsociology', 'LSEStatistics']

In [10]:
import pandas as pd

dept_usernames = pd.DataFrame({
    'Department': dept_list,
    'Twitter username': usernames
})

dept_usernames['Department'] = dept_usernames['Department'].str.replace('Department of ', '')
dept_usernames

,Department,Twitter username
0,Accounting,LSE_Accounting
1,Anthropology,LSEAnthropology
2,Economics,LSEEcon
3,Economic History,LSEEcHist
4,Finance,LSEfinance
5,Gender Studies,LSEGenderTweet
6,Geography and Environment,LSEGeography
7,Government,LSEGovernment
8,Health Policy,LSEHealthPolicy
9,International Development,LSE_ID


Next, we start identifying what kind of tweets we would like to look at for each department. We want to find out which department has better engagement overall and see what factors contribute to its success. The first thing we will look at is each department's follower counts. The next step is to find out the overall engagement using the metrics of likes, retweets, replies, and quotes. In the next step, we will look at comments (which may be limited). All the data analysis will only go back to the last 100 tweets posted by each account because this is the limit that Twitter allows.

Because of the restrictions of the Twitter developer account, the engagement counts are public metrics. If there were no restrictions, I would add more metrics like impression count, total views, and profile views.

In [11]:
with open('keys.json') as f:
    keys = json.load(f)

bearer_token = keys['twitter']['bearer_token']
headers = {
    'Authorization': f"Bearer {bearer_token}"
}

In [12]:
r = requests.get('https://api.twitter.com/2/users/by?usernames=LSE_Accounting,LSEAnthropology,LSEEcon,LSEEcHist,LSEfinance,LSEGenderTweet,LSEGeography,LSEGovernment,LSEHealthPolicy,LSE_ID,lsehistory,LSEIRDept,lselangcentre,LSELaw,LSEManagement,LSEMaths,MediaLSE,MethodologyLSE,LSEPhilosophy,LSEBehavioural,LSESocialPolicy,LSEsociology,LSEStatistics&user.fields=public_metrics', headers=headers)
r.text

'{"data":[{"id":"4900666161","username":"LSE_Accounting","public_metrics":{"followers_count":2513,"following_count":119,"tweet_count":494,"listed_count":25},"name":"LSE Accounting"},{"id":"850888387","username":"LSEAnthropology","public_metrics":{"followers_count":6642,"following_count":93,"tweet_count":899,"listed_count":106},"name":"LSE Anthropology"},{"id":"1200727465","username":"LSEEcon","public_metrics":{"followers_count":35620,"following_count":636,"tweet_count":10179,"listed_count":568},"name":"LSE Department of Economics"},{"id":"224639696","username":"LSEEcHist","public_metrics":{"followers_count":3874,"following_count":304,"tweet_count":1745,"listed_count":99},"name":"LSE Economic History"},{"id":"972257048","username":"LSEfinance","public_metrics":{"followers_count":2478,"following_count":179,"tweet_count":605,"listed_count":51},"name":"LSE Finance"},{"id":"189090262","username":"LSEGenderTweet","public_metrics":{"followers_count":19644,"following_count":2622,"tweet_count":

In [13]:
ids = json.loads(r.text)['data']

dept_ids = []
dept_followers = []

for i in range(0, len(ids)):
    idno = ids[i]['id']
    followers = ids[i]['public_metrics']['followers_count']
    dept_ids.append(idno)
    dept_followers.append(followers)

dept_usernames['Twitter ID'] = dept_ids
dept_usernames['Follower count'] = dept_followers

In [14]:
dept_usernames

,Department,Twitter username,Twitter ID,Follower count
0,Accounting,LSE_Accounting,4900666161,2513
1,Anthropology,LSEAnthropology,850888387,6642
2,Economics,LSEEcon,1200727465,35620
3,Economic History,LSEEcHist,224639696,3874
4,Finance,LSEfinance,972257048,2478
5,Gender Studies,LSEGenderTweet,189090262,19644
6,Geography and Environment,LSEGeography,240262055,12553
7,Government,LSEGovernment,303823238,24726
8,Health Policy,LSEHealthPolicy,472009727,7635
9,International Development,LSE_ID,317018025,12130


To start, I will extract the number of engagements for the last 100 tweets of every LSE department on Twitter.

In [15]:
#Accounting department
r = requests.get('https://api.twitter.com/2/users/4900666161/tweets?expansions=attachments.poll_ids,attachments.media_keys&tweet.fields=public_metrics&poll.fields=end_datetime&max_results=100', headers=headers)
acc = json.loads(r.text)

acc_dept_tweets = acc['data']
acc_tweetid = []; acc_retweet = []; acc_reply = []; acc_like = []; acc_quote = []; dept = []

for i in range(0, len(acc_dept_tweets)):
    dept.append('Accounting')
    acc_tweetid.append(acc_dept_tweets[i]['id'])
    acc_retweet.append(acc_dept_tweets[i]['public_metrics']['retweet_count'])
    acc_reply.append(acc_dept_tweets[i]['public_metrics']['reply_count'])
    acc_like.append(acc_dept_tweets[i]['public_metrics']['like_count'])
    acc_quote.append(acc_dept_tweets[i]['public_metrics']['quote_count'])
    
acc_twitter = pd.DataFrame({
    'Twitter Id': acc_tweetid,
    'Department': dept,
    'Retweet count': acc_retweet,
    'Reply count': acc_reply,
    'Like count': acc_like,
    'Quote count': acc_quote
})

In [16]:
#Anthropology department
r = requests.get('https://api.twitter.com/2/users/850888387/tweets?expansions=attachments.poll_ids,attachments.media_keys&tweet.fields=public_metrics&poll.fields=end_datetime&max_results=100', headers=headers)
anth = json.loads(r.text)

anth_dept_tweets = anth['data']
anth_tweetid = []; anth_retweet = []; anth_reply = []; anth_like = []; anth_quote = []; dept = []

for i in range(0, len(anth_dept_tweets)):
    dept.append('Anthropology')
    anth_tweetid.append(anth_dept_tweets[i]['id'])
    anth_retweet.append(anth_dept_tweets[i]['public_metrics']['retweet_count'])
    anth_reply.append(anth_dept_tweets[i]['public_metrics']['reply_count'])
    anth_like.append(anth_dept_tweets[i]['public_metrics']['like_count'])
    anth_quote.append(anth_dept_tweets[i]['public_metrics']['quote_count'])
    
anth_twitter = pd.DataFrame({
    'Twitter Id': anth_tweetid,
    'Department': dept,
    'Retweet count': anth_retweet,
    'Reply count': anth_reply,
    'Like count': anth_like,
    'Quote count': anth_quote
})

In [17]:
#Economics department
r = requests.get('https://api.twitter.com/2/users/1200727465/tweets?expansions=attachments.poll_ids,attachments.media_keys&tweet.fields=public_metrics&poll.fields=end_datetime&max_results=100', headers=headers)
econ = json.loads(r.text)

econ_dept_tweets = econ['data']
econ_tweetid = []; econ_retweet = []; econ_reply = []; econ_like = []; econ_quote = []; dept = []

for i in range(0, len(econ_dept_tweets)):
    dept.append('Economics')
    econ_tweetid.append(econ_dept_tweets[i]['id'])
    econ_retweet.append(econ_dept_tweets[i]['public_metrics']['retweet_count'])
    econ_reply.append(econ_dept_tweets[i]['public_metrics']['reply_count'])
    econ_like.append(econ_dept_tweets[i]['public_metrics']['like_count'])
    econ_quote.append(econ_dept_tweets[i]['public_metrics']['quote_count'])
    
econ_twitter = pd.DataFrame({
    'Twitter Id': econ_tweetid,
    'Department': dept,
    'Retweet count': econ_retweet,
    'Reply count': econ_reply,
    'Like count': econ_like,
    'Quote count': econ_quote
})

In [18]:
#Economics history department
r = requests.get('https://api.twitter.com/2/users/224639696/tweets?expansions=attachments.poll_ids,attachments.media_keys&tweet.fields=public_metrics&poll.fields=end_datetime&max_results=100', headers=headers)
econhist = json.loads(r.text)

econhist_dept_tweets = econhist['data']
econhist_tweetid = []; econhist_retweet = []; econhist_reply = []; econhist_like = []; econhist_quote = []; dept = []

for i in range(0, len(econhist_dept_tweets)):
    dept.append('Economics History')
    econhist_tweetid.append(econhist_dept_tweets[i]['id'])
    econhist_retweet.append(econhist_dept_tweets[i]['public_metrics']['retweet_count'])
    econhist_reply.append(econhist_dept_tweets[i]['public_metrics']['reply_count'])
    econhist_like.append(econhist_dept_tweets[i]['public_metrics']['like_count'])
    econhist_quote.append(econhist_dept_tweets[i]['public_metrics']['quote_count'])
    
econhist_twitter = pd.DataFrame({
    'Twitter Id': econhist_tweetid,
    'Department': dept,
    'Retweet count': econhist_retweet,
    'Reply count': econhist_reply,
    'Like count': econhist_like,
    'Quote count': econhist_quote
})

In [19]:
#Finance department
r = requests.get('https://api.twitter.com/2/users/972257048/tweets?expansions=attachments.poll_ids,attachments.media_keys&tweet.fields=public_metrics&poll.fields=end_datetime&max_results=100', headers=headers)
finance = json.loads(r.text)

finance_dept_tweets = finance['data']
finance_tweetid = []; finance_retweet = []; finance_reply = []; finance_like = []; finance_quote = []; dept = []

for i in range(0, len(finance_dept_tweets)):
    dept.append('Finance')
    finance_tweetid.append(finance_dept_tweets[i]['id'])
    finance_retweet.append(finance_dept_tweets[i]['public_metrics']['retweet_count'])
    finance_reply.append(finance_dept_tweets[i]['public_metrics']['reply_count'])
    finance_like.append(finance_dept_tweets[i]['public_metrics']['like_count'])
    finance_quote.append(finance_dept_tweets[i]['public_metrics']['quote_count'])
    
finance_twitter = pd.DataFrame({
    'Twitter Id': finance_tweetid,
    'Department': dept,
    'Retweet count': finance_retweet,
    'Reply count': finance_reply,
    'Like count': finance_like,
    'Quote count': finance_quote
})

In [20]:
#Gender studies department
r = requests.get('https://api.twitter.com/2/users/189090262/tweets?expansions=attachments.poll_ids,attachments.media_keys&tweet.fields=public_metrics&poll.fields=end_datetime&max_results=100', headers=headers)
gender = json.loads(r.text)

gender_dept_tweets = gender['data']
gender_tweetid = []; gender_retweet = []; gender_reply = []; gender_like = []; gender_quote = []; dept = []

for i in range(0, len(gender_dept_tweets)):
    dept.append('Gender Studies')
    gender_tweetid.append(gender_dept_tweets[i]['id'])
    gender_retweet.append(gender_dept_tweets[i]['public_metrics']['retweet_count'])
    gender_reply.append(gender_dept_tweets[i]['public_metrics']['reply_count'])
    gender_like.append(gender_dept_tweets[i]['public_metrics']['like_count'])
    gender_quote.append(gender_dept_tweets[i]['public_metrics']['quote_count'])
    
gender_twitter = pd.DataFrame({
    'Twitter Id': gender_tweetid,
    'Department': dept,
    'Retweet count': gender_retweet,
    'Reply count': gender_reply,
    'Like count': gender_like,
    'Quote count': gender_quote
})

In [21]:
#Geography department
r = requests.get('https://api.twitter.com/2/users/240262055/tweets?expansions=attachments.poll_ids,attachments.media_keys&tweet.fields=public_metrics&poll.fields=end_datetime&max_results=100', headers=headers)
geo = json.loads(r.text)

geo_dept_tweets = geo['data']
geo_tweetid = []; geo_retweet = []; geo_reply = []; geo_like = []; geo_quote = []; dept = []

for i in range(0, len(geo_dept_tweets)):
    dept.append('Geography and Environment')
    geo_tweetid.append(geo_dept_tweets[i]['id'])
    geo_retweet.append(geo_dept_tweets[i]['public_metrics']['retweet_count'])
    geo_reply.append(geo_dept_tweets[i]['public_metrics']['reply_count'])
    geo_like.append(geo_dept_tweets[i]['public_metrics']['like_count'])
    geo_quote.append(geo_dept_tweets[i]['public_metrics']['quote_count'])
    
geo_twitter = pd.DataFrame({
    'Twitter Id': geo_tweetid,
    'Department': dept,
    'Retweet count': geo_retweet,
    'Reply count': geo_reply,
    'Like count': geo_like,
    'Quote count': geo_quote
})

In [22]:
#Government department
r = requests.get('https://api.twitter.com/2/users/303823238/tweets?expansions=attachments.poll_ids,attachments.media_keys&tweet.fields=public_metrics&poll.fields=end_datetime&max_results=100', headers=headers)
gov = json.loads(r.text)

gov_dept_tweets = gov['data']
gov_tweetid = []; gov_retweet = []; gov_reply = []; gov_like = []; gov_quote = []; dept = []

for i in range(0, len(gov_dept_tweets)):
    dept.append('Government')
    gov_tweetid.append(gov_dept_tweets[i]['id'])
    gov_retweet.append(gov_dept_tweets[i]['public_metrics']['retweet_count'])
    gov_reply.append(gov_dept_tweets[i]['public_metrics']['reply_count'])
    gov_like.append(gov_dept_tweets[i]['public_metrics']['like_count'])
    gov_quote.append(gov_dept_tweets[i]['public_metrics']['quote_count'])
    
gov_twitter = pd.DataFrame({
    'Twitter Id': gov_tweetid,
    'Department': dept,
    'Retweet count': gov_retweet,
    'Reply count': gov_reply,
    'Like count': gov_like,
    'Quote count': gov_quote
})

In [23]:
#Health Policy Department
r = requests.get('https://api.twitter.com/2/users/472009727/tweets?expansions=attachments.poll_ids,attachments.media_keys&tweet.fields=public_metrics&poll.fields=end_datetime&max_results=100', headers=headers)
hp = json.loads(r.text)

hp_dept_tweets = hp['data']
hp_tweetid = []; hp_retweet = []; hp_reply = []; hp_like = []; hp_quote = []; dept = []

for i in range(0, len(hp_dept_tweets)):
    dept.append('Health Policy')
    hp_tweetid.append(hp_dept_tweets[i]['id'])
    hp_retweet.append(hp_dept_tweets[i]['public_metrics']['retweet_count'])
    hp_reply.append(hp_dept_tweets[i]['public_metrics']['reply_count'])
    hp_like.append(hp_dept_tweets[i]['public_metrics']['like_count'])
    hp_quote.append(hp_dept_tweets[i]['public_metrics']['quote_count'])
    
hp_twitter = pd.DataFrame({
    'Twitter Id': hp_tweetid,
    'Department': dept,
    'Retweet count': hp_retweet,
    'Reply count': hp_reply,
    'Like count': hp_like,
    'Quote count': hp_quote
})

In [24]:
#International Development Department
r = requests.get('https://api.twitter.com/2/users/317018025/tweets?expansions=attachments.poll_ids,attachments.media_keys&tweet.fields=public_metrics&poll.fields=end_datetime&max_results=100', headers=headers)
id = json.loads(r.text)

id_dept_tweets = id['data']
id_tweetid = []; id_retweet = []; id_reply = []; id_like = []; id_quote = []; dept = []

for i in range(0, len(id_dept_tweets)):
    dept.append('International Development')
    id_tweetid.append(id_dept_tweets[i]['id'])
    id_retweet.append(id_dept_tweets[i]['public_metrics']['retweet_count'])
    id_reply.append(id_dept_tweets[i]['public_metrics']['reply_count'])
    id_like.append(id_dept_tweets[i]['public_metrics']['like_count'])
    id_quote.append(id_dept_tweets[i]['public_metrics']['quote_count'])
    
id_twitter = pd.DataFrame({
    'Twitter Id': id_tweetid,
    'Department': dept,
    'Retweet count': id_retweet,
    'Reply count': id_reply,
    'Like count': id_like,
    'Quote count': id_quote
})

In [25]:
#International History Department
r = requests.get('https://api.twitter.com/2/users/253471591/tweets?expansions=attachments.poll_ids,attachments.media_keys&tweet.fields=public_metrics&poll.fields=end_datetime&max_results=100', headers=headers)
ih = json.loads(r.text)

ih_dept_tweets = ih['data']
ih_tweetid = []; ih_retweet = []; ih_reply = []; ih_like = []; ih_quote = []; dept = []

for i in range(0, len(ih_dept_tweets)):
    dept.append('International History')
    ih_tweetid.append(ih_dept_tweets[i]['id'])
    ih_retweet.append(ih_dept_tweets[i]['public_metrics']['retweet_count'])
    ih_reply.append(ih_dept_tweets[i]['public_metrics']['reply_count'])
    ih_like.append(ih_dept_tweets[i]['public_metrics']['like_count'])
    ih_quote.append(ih_dept_tweets[i]['public_metrics']['quote_count'])
    
ih_twitter = pd.DataFrame({
    'Twitter Id': ih_tweetid,
    'Department': dept,
    'Retweet count': ih_retweet,
    'Reply count': ih_reply,
    'Like count': ih_like,
    'Quote count': ih_quote
})

In [26]:
#International Relations Department
r = requests.get('https://api.twitter.com/2/users/237225532/tweets?expansions=attachments.poll_ids,attachments.media_keys&tweet.fields=public_metrics&poll.fields=end_datetime&max_results=100', headers=headers)
ir = json.loads(r.text)

ir_dept_tweets = ir['data']
ir_tweetid = []; ir_retweet = []; ir_reply = []; ir_like = []; ir_quote = []; dept = []

for i in range(0, len(ir_dept_tweets)):
    dept.append('International Relations')
    ir_tweetid.append(ir_dept_tweets[i]['id'])
    ir_retweet.append(ir_dept_tweets[i]['public_metrics']['retweet_count'])
    ir_reply.append(ir_dept_tweets[i]['public_metrics']['reply_count'])
    ir_like.append(ir_dept_tweets[i]['public_metrics']['like_count'])
    ir_quote.append(ir_dept_tweets[i]['public_metrics']['quote_count'])
    
ir_twitter = pd.DataFrame({
    'Twitter Id': ir_tweetid,
    'Department': dept,
    'Retweet count': ir_retweet,
    'Reply count': ir_reply,
    'Like count': ir_like,
    'Quote count': ir_quote
})

In [27]:
#Language Centre Department
r = requests.get('https://api.twitter.com/2/users/179888345/tweets?expansions=attachments.poll_ids,attachments.media_keys&tweet.fields=public_metrics&poll.fields=end_datetime&max_results=100', headers=headers)
lang = json.loads(r.text)

lang_dept_tweets = lang['data']
lang_tweetid = []; lang_retweet = []; lang_reply = []; lang_like = []; lang_quote = []; dept = []

for i in range(0, len(lang_dept_tweets)):
    dept.append('Language Centre')
    lang_tweetid.append(lang_dept_tweets[i]['id'])
    lang_retweet.append(lang_dept_tweets[i]['public_metrics']['retweet_count'])
    lang_reply.append(lang_dept_tweets[i]['public_metrics']['reply_count'])
    lang_like.append(lang_dept_tweets[i]['public_metrics']['like_count'])
    lang_quote.append(lang_dept_tweets[i]['public_metrics']['quote_count'])
    
lang_twitter = pd.DataFrame({
    'Twitter Id': lang_tweetid,
    'Department': dept,
    'Retweet count': lang_retweet,
    'Reply count': lang_reply,
    'Like count': lang_like,
    'Quote count': lang_quote
})

In [28]:
#Law Department
r = requests.get('https://api.twitter.com/2/users/532172035/tweets?expansions=attachments.poll_ids,attachments.media_keys&tweet.fields=public_metrics&poll.fields=end_datetime&max_results=100', headers=headers)
law = json.loads(r.text)

law_dept_tweets = law['data']
law_tweetid = []; law_retweet = []; law_reply = []; law_like = []; law_quote = []; dept = []

for i in range(0, len(law_dept_tweets)):
    dept.append('LSE Law School')
    law_tweetid.append(law_dept_tweets[i]['id'])
    law_retweet.append(law_dept_tweets[i]['public_metrics']['retweet_count'])
    law_reply.append(law_dept_tweets[i]['public_metrics']['reply_count'])
    law_like.append(law_dept_tweets[i]['public_metrics']['like_count'])
    law_quote.append(law_dept_tweets[i]['public_metrics']['quote_count'])
    
law_twitter = pd.DataFrame({
    'Twitter Id': law_tweetid,
    'Department': dept,
    'Retweet count': law_retweet,
    'Reply count': law_reply,
    'Like count': law_like,
    'Quote count': law_quote
})

In [29]:
#Management Department
r = requests.get('https://api.twitter.com/2/users/26465977/tweets?expansions=attachments.poll_ids,attachments.media_keys&tweet.fields=public_metrics&poll.fields=end_datetime&max_results=100', headers=headers)
man = json.loads(r.text)

man_dept_tweets = man['data']
man_tweetid = []; man_retweet = []; man_reply = []; man_like = []; man_quote = []; dept = []

for i in range(0, len(man_dept_tweets)):
    dept.append('Management')
    man_tweetid.append(man_dept_tweets[i]['id'])
    man_retweet.append(man_dept_tweets[i]['public_metrics']['retweet_count'])
    man_reply.append(man_dept_tweets[i]['public_metrics']['reply_count'])
    man_like.append(man_dept_tweets[i]['public_metrics']['like_count'])
    man_quote.append(man_dept_tweets[i]['public_metrics']['quote_count'])
    
man_twitter = pd.DataFrame({
    'Twitter Id': man_tweetid,
    'Department': dept,
    'Retweet count': man_retweet,
    'Reply count': man_reply,
    'Like count': man_like,
    'Quote count': man_quote
})

In [30]:
#Mathematics Department
r = requests.get('https://api.twitter.com/2/users/3044880371/tweets?expansions=attachments.poll_ids,attachments.media_keys&tweet.fields=public_metrics&poll.fields=end_datetime&max_results=100', headers=headers)
math = json.loads(r.text)

math_dept_tweets = math['data']
math_tweetid = []; math_retweet = []; math_reply = []; math_like = []; math_quote = []; dept = []

for i in range(0, len(math_dept_tweets)):
    dept.append('Mathematics')
    math_tweetid.append(math_dept_tweets[i]['id'])
    math_retweet.append(math_dept_tweets[i]['public_metrics']['retweet_count'])
    math_reply.append(math_dept_tweets[i]['public_metrics']['reply_count'])
    math_like.append(math_dept_tweets[i]['public_metrics']['like_count'])
    math_quote.append(math_dept_tweets[i]['public_metrics']['quote_count'])
    
math_twitter = pd.DataFrame({
    'Twitter Id': math_tweetid,
    'Department': dept,
    'Retweet count': math_retweet,
    'Reply count': math_reply,
    'Like count': math_like,
    'Quote count': math_quote
})

In [31]:
#Media and Communications Department
r = requests.get('https://api.twitter.com/2/users/207534677/tweets?expansions=attachments.poll_ids,attachments.media_keys&tweet.fields=public_metrics&poll.fields=end_datetime&max_results=100', headers=headers)
mc = json.loads(r.text)

mc_dept_tweets = mc['data']
mc_tweetid = []; mc_retweet = []; mc_reply = []; mc_like = []; mc_quote = []; dept = []

for i in range(0, len(mc_dept_tweets)):
    dept.append('Media and Communications')
    mc_tweetid.append(mc_dept_tweets[i]['id'])
    mc_retweet.append(mc_dept_tweets[i]['public_metrics']['retweet_count'])
    mc_reply.append(mc_dept_tweets[i]['public_metrics']['reply_count'])
    mc_like.append(mc_dept_tweets[i]['public_metrics']['like_count'])
    mc_quote.append(mc_dept_tweets[i]['public_metrics']['quote_count'])
    
mc_twitter = pd.DataFrame({
    'Twitter Id': mc_tweetid,
    'Department': dept,
    'Retweet count': mc_retweet,
    'Reply count': mc_reply,
    'Like count': mc_like,
    'Quote count': mc_quote
})

In [32]:
#Methodology Department
r = requests.get('https://api.twitter.com/2/users/86921024/tweets?expansions=attachments.poll_ids,attachments.media_keys&tweet.fields=public_metrics&poll.fields=end_datetime&max_results=100', headers=headers)
met = json.loads(r.text)

met_dept_tweets = met['data']
met_tweetid = []; met_retweet = []; met_reply = []; met_like = []; met_quote = []; dept = []

for i in range(0, len(met_dept_tweets)):
    dept.append('Methodology')
    met_tweetid.append(met_dept_tweets[i]['id'])
    met_retweet.append(met_dept_tweets[i]['public_metrics']['retweet_count'])
    met_reply.append(met_dept_tweets[i]['public_metrics']['reply_count'])
    met_like.append(met_dept_tweets[i]['public_metrics']['like_count'])
    met_quote.append(met_dept_tweets[i]['public_metrics']['quote_count'])
    
met_twitter = pd.DataFrame({
    'Twitter Id': met_tweetid,
    'Department': dept,
    'Retweet count': met_retweet,
    'Reply count': met_reply,
    'Like count': met_like,
    'Quote count': met_quote
})

In [33]:
#Philosophy Department
r = requests.get('https://api.twitter.com/2/users/904251031/tweets?expansions=attachments.poll_ids,attachments.media_keys&tweet.fields=public_metrics&poll.fields=end_datetime&max_results=100', headers=headers)
phil = json.loads(r.text)

phil_dept_tweets = phil['data']
phil_tweetid = []; phil_retweet = []; phil_reply = []; phil_like = []; phil_quote = []; dept = []

for i in range(0, len(phil_dept_tweets)):
    dept.append('Philosophy, Logic and Scientific Method')
    phil_tweetid.append(phil_dept_tweets[i]['id'])
    phil_retweet.append(phil_dept_tweets[i]['public_metrics']['retweet_count'])
    phil_reply.append(phil_dept_tweets[i]['public_metrics']['reply_count'])
    phil_like.append(phil_dept_tweets[i]['public_metrics']['like_count'])
    phil_quote.append(phil_dept_tweets[i]['public_metrics']['quote_count'])
    
phil_twitter = pd.DataFrame({
    'Twitter Id': phil_tweetid,
    'Department': dept,
    'Retweet count': phil_retweet,
    'Reply count': phil_reply,
    'Like count': phil_like,
    'Quote count': phil_quote
})

In [34]:
#Pscyhology Department
r = requests.get('https://api.twitter.com/2/users/1965000560/tweets?expansions=attachments.poll_ids,attachments.media_keys&tweet.fields=public_metrics&poll.fields=end_datetime&max_results=100', headers=headers)
psych = json.loads(r.text)

psych_dept_tweets = psych['data']
psych_tweetid = []; psych_retweet = []; psych_reply = []; psych_like = []; psych_quote = []; dept = []

for i in range(0, len(psych_dept_tweets)):
    dept.append('Psychological and Behavioural Science')
    psych_tweetid.append(psych_dept_tweets[i]['id'])
    psych_retweet.append(psych_dept_tweets[i]['public_metrics']['retweet_count'])
    psych_reply.append(psych_dept_tweets[i]['public_metrics']['reply_count'])
    psych_like.append(psych_dept_tweets[i]['public_metrics']['like_count'])
    psych_quote.append(psych_dept_tweets[i]['public_metrics']['quote_count'])
    
psych_twitter = pd.DataFrame({
    'Twitter Id': psych_tweetid,
    'Department': dept,
    'Retweet count': psych_retweet,
    'Reply count': psych_reply,
    'Like count': psych_like,
    'Quote count': psych_quote
})

In [35]:
#Social Policy Department
r = requests.get('https://api.twitter.com/2/users/2472172578/tweets?expansions=attachments.poll_ids,attachments.media_keys&tweet.fields=public_metrics&poll.fields=end_datetime&max_results=100', headers=headers)
sp = json.loads(r.text)

sp_dept_tweets = sp['data']
sp_tweetid = []; sp_retweet = []; sp_reply = []; sp_like = []; sp_quote = []; dept = []

for i in range(0, len(sp_dept_tweets)):
    dept.append('Social Policy')
    sp_tweetid.append(sp_dept_tweets[i]['id'])
    sp_retweet.append(sp_dept_tweets[i]['public_metrics']['retweet_count'])
    sp_reply.append(sp_dept_tweets[i]['public_metrics']['reply_count'])
    sp_like.append(sp_dept_tweets[i]['public_metrics']['like_count'])
    sp_quote.append(sp_dept_tweets[i]['public_metrics']['quote_count'])
    
sp_twitter = pd.DataFrame({
    'Twitter Id': sp_tweetid,
    'Department': dept,
    'Retweet count': sp_retweet,
    'Reply count': sp_reply,
    'Like count': sp_like,
    'Quote count': sp_quote
})

In [36]:
#Sociology Department
r = requests.get('https://api.twitter.com/2/users/1671486960/tweets?expansions=attachments.poll_ids,attachments.media_keys&tweet.fields=public_metrics&poll.fields=end_datetime&max_results=100', headers=headers)
socio = json.loads(r.text)

socio_dept_tweets = socio['data']
socio_tweetid = []; socio_retweet = []; socio_reply = []; socio_like = []; socio_quote = []; dept = []

for i in range(0, len(socio_dept_tweets)):
    dept.append('Sociology')
    socio_tweetid.append(socio_dept_tweets[i]['id'])
    socio_retweet.append(socio_dept_tweets[i]['public_metrics']['retweet_count'])
    socio_reply.append(socio_dept_tweets[i]['public_metrics']['reply_count'])
    socio_like.append(socio_dept_tweets[i]['public_metrics']['like_count'])
    socio_quote.append(socio_dept_tweets[i]['public_metrics']['quote_count'])
    
socio_twitter = pd.DataFrame({
    'Twitter Id': socio_tweetid,
    'Department': dept,
    'Retweet count': socio_retweet,
    'Reply count': socio_reply,
    'Like count': socio_like,
    'Quote count': socio_quote
})

In [37]:
#Statistics Department
r = requests.get('https://api.twitter.com/2/users/420282103/tweets?expansions=attachments.poll_ids,attachments.media_keys&tweet.fields=public_metrics&poll.fields=end_datetime&max_results=100', headers=headers)
stats = json.loads(r.text)

stats_dept_tweets = stats['data']
stats_tweetid = []; stats_retweet = []; stats_reply = []; stats_like = []; stats_quote = []; dept = []

for i in range(0, len(stats_dept_tweets)):
    dept.append('Statistics')
    stats_tweetid.append(stats_dept_tweets[i]['id'])
    stats_retweet.append(stats_dept_tweets[i]['public_metrics']['retweet_count'])
    stats_reply.append(stats_dept_tweets[i]['public_metrics']['reply_count'])
    stats_like.append(stats_dept_tweets[i]['public_metrics']['like_count'])
    stats_quote.append(stats_dept_tweets[i]['public_metrics']['quote_count'])
    
stats_twitter = pd.DataFrame({
    'Twitter Id': stats_tweetid,
    'Department': dept,
    'Retweet count': stats_retweet,
    'Reply count': stats_reply,
    'Like count': stats_like,
    'Quote count': stats_quote
})

In [227]:
tweets_stats = pd.concat([acc_twitter, anth_twitter, econ_twitter, econhist_twitter,
         finance_twitter, gender_twitter, geo_twitter, gov_twitter,
         hp_twitter, id_twitter, ih_twitter, ir_twitter, lang_twitter,
         law_twitter, man_twitter, math_twitter, mc_twitter, met_twitter,
         phil_twitter, psych_twitter, sp_twitter, socio_twitter, stats_twitter])

tweets_stats 

,Twitter Id,Department,Retweet count,Reply count,Like count,Quote count
0,1506004918016094208,Accounting,1,0,0,0
1,1502055596488540164,Accounting,1,0,0,0
2,1501594873509691393,Accounting,2,0,4,0
3,1498763109401612296,Accounting,3,0,0,0
4,1498762781457321986,Accounting,1,0,0,0
...,...,...,...,...,...,...
95,1485971876207906818,Statistics,3,0,0,0
96,1485949504906051589,Statistics,0,0,2,0
97,1484103476263194624,Statistics,0,0,1,0
98,1483801755452387329,Statistics,0,0,0,0


Earlier, we found the follower count of each department's social media. Now that we have both the followers and engagement totals, we can calculate the engagement ratio for each account. For engagement ratio, we can take the total engagements(retweets, replies, likes and quotes) and divide them by followers.

In [228]:
dept_eng = tweets_stats.groupby(['Department'], as_index=False).sum()
dept_eng['Follower count'] = dept_usernames['Follower count']
dept_eng['Total engagement'] = dept_eng['Retweet count']+dept_eng['Reply count']+dept_eng['Like count']+dept_eng['Quote count']

In [229]:
dept_eng['Engagement ratio'] = dept_eng['Total engagement']/dept_eng['Follower count']
dept_eng

,Department,Retweet count,Reply count,Like count,Quote count,Follower count,Total engagement,Engagement ratio
0,Accounting,288,1,94,3,2513,386,0.153601
1,Anthropology,673,4,388,16,6642,1081,0.162752
2,Economics,368,6,283,8,35620,665,0.018669
3,Economics History,144,66,346,13,3874,569,0.146877
4,Finance,391,1,38,3,2478,433,0.174738
5,Gender Studies,1592,10,308,19,19644,1929,0.098198
6,Geography and Environment,471,4,267,13,12553,755,0.060145
7,Government,377,7,131,7,24726,522,0.021111
8,Health Policy,330,22,415,13,7635,780,0.102161
9,International Development,290,5,186,15,12130,496,0.040890


The last thing we need to collect is information about the mentions of the department's accounts. After extracting this, I will also perform sentiment analyses for the mentions. Once the sentiment analysis is complete, we can count how many mentions are positive.

In [159]:
#Accounting department
r = requests.get('https://api.twitter.com/2/users/4900666161/mentions?expansions=author_id&max_results=100', headers=headers)
acc_m = json.loads(r.text)['data']

dept_id = []; acc_m_user = []; acc_m_content = []; dept = []
for i in range(0, len(acc_m)):
    dept_id.append('4900666161')
    dept.append('Accounting')
    user = acc_m[i]['author_id']
    content = acc_m[i]['text']
    acc_m_user.append(user)
    acc_m_content.append(content)
    
acc_mentions = pd.DataFrame({
    'User ID': acc_m_user,
    'Department': dept,
    'Department ID': dept_id,
    'Tweet content': acc_m_content
})

In [160]:
#Anthropology department
r = requests.get('https://api.twitter.com/2/users/850888387/mentions?expansions=author_id&max_results=100', headers=headers)
anth_m = json.loads(r.text)['data']

dept_id = []; anth_m_user = []; anth_m_content = []; dept = []
for i in range(0, len(anth_m)):
    dept_id.append('850888387')
    dept.append('Anthropology')
    user = anth_m[i]['author_id']
    content = anth_m[i]['text']
    anth_m_user.append(user)
    anth_m_content.append(content)
    
anth_mentions = pd.DataFrame({
    'User ID': anth_m_user,
    'Department': dept,
    'Department ID': dept_id,
    'Tweet content': anth_m_content
})

In [161]:
#Economics department
r = requests.get('https://api.twitter.com/2/users/1200727465/mentions?expansions=author_id&max_results=100', headers=headers)
econ_m = json.loads(r.text)['data']

dept_id = []; econ_m_user = []; econ_m_content = []; dept = []
for i in range(0, len(econ_m)):
    dept_id.append('1200727465')
    dept.append('Economics')
    user = econ_m[i]['author_id']
    content = econ_m[i]['text']
    econ_m_user.append(user)
    econ_m_content.append(content)
    
econ_mentions = pd.DataFrame({
    'User ID': econ_m_user,
    'Department': dept,
    'Department ID': dept_id,
    'Tweet content': econ_m_content
})

In [162]:
#Economic History department
r = requests.get('https://api.twitter.com/2/users/224639696/mentions?expansions=author_id&max_results=100', headers=headers)
econhist_m = json.loads(r.text)['data']

dept_id = []; econhist_m_user = []; econhist_m_content = []; dept = []
for i in range(0, len(econhist_m)):
    dept_id.append('224639696')
    dept.append('Economic History')
    user = econhist_m[i]['author_id']
    content = econhist_m[i]['text']
    econhist_m_user.append(user)
    econhist_m_content.append(content)
    
econhist_mentions = pd.DataFrame({
    'User ID': econhist_m_user,
    'Department': dept,
    'Department ID': dept_id,
    'Tweet content': econhist_m_content
})

In [163]:
#Finance Department
r = requests.get('https://api.twitter.com/2/users/972257048/mentions?expansions=author_id&max_results=100', headers=headers)
finance_m = json.loads(r.text)['data']

dept_id = []; finance_m_user = []; finance_m_content = []; dept = []
for i in range(0, len(finance_m)):
    dept_id.append('972257048')
    dept.append('Finance')
    user = finance_m[i]['author_id']
    content = finance_m[i]['text']
    finance_m_user.append(user)
    finance_m_content.append(content)
    
finance_mentions = pd.DataFrame({
    'User ID': finance_m_user,
    'Department': dept,
    'Department ID': dept_id,
    'Tweet content': finance_m_content
})

In [164]:
#Gender Studies Department
r = requests.get('https://api.twitter.com/2/users/189090262/mentions?expansions=author_id&max_results=100', headers=headers)
gender_m = json.loads(r.text)['data']

dept_id = []; gender_m_user = []; gender_m_content = []; dept = []
for i in range(0, len(gender_m)):
    dept_id.append('189090262')
    dept.append('Gender Studies')
    user = gender_m[i]['author_id']
    content = gender_m[i]['text']
    gender_m_user.append(user)
    gender_m_content.append(content)
    
gender_mentions = pd.DataFrame({
    'User ID': gender_m_user,
    'Department': dept,
    'Department ID': dept_id,
    'Tweet content': gender_m_content
})

In [165]:
#Geography department
r = requests.get('https://api.twitter.com/2/users/240262055/mentions?expansions=author_id&max_results=100', headers=headers)
geo_m = json.loads(r.text)['data']

dept_id = []; geo_m_user = []; geo_m_content = []; dept = []
for i in range(0, len(geo_m)):
    dept_id.append('240262055')
    dept.append('Geography and Evironment')
    user = geo_m[i]['author_id']
    content = geo_m[i]['text']
    geo_m_user.append(user)
    geo_m_content.append(content)
    
geo_mentions = pd.DataFrame({
    'User ID': geo_m_user,
    'Department': dept,
    'Department ID': dept_id,
    'Tweet content': geo_m_content
})

In [166]:
#Government Department
r = requests.get('https://api.twitter.com/2/users/303823238/mentions?expansions=author_id&max_results=100', headers=headers)
gov_m = json.loads(r.text)['data']

dept_id = []; gov_m_user = []; gov_m_content = []; dept = []
for i in range(0, len(gov_m)):
    dept_id.append('303823238')
    dept.append('Government')
    user = gov_m[i]['author_id']
    content = gov_m[i]['text']
    gov_m_user.append(user)
    gov_m_content.append(content)
    
gov_mentions = pd.DataFrame({
    'User ID': gov_m_user,
    'Department': dept,
    'Department ID': dept_id,
    'Tweet content': gov_m_content
})

In [167]:
#Health Policy Department
r = requests.get('https://api.twitter.com/2/users/472009727/mentions?expansions=author_id&max_results=100', headers=headers)
hp_m = json.loads(r.text)['data']

dept_id = []; hp_m_user = []; hp_m_content = []; dept = []
for i in range(0, len(hp_m)):
    dept_id.append('472009727')
    dept.append('Health Policy')
    user = hp_m[i]['author_id']
    content = hp_m[i]['text']
    hp_m_user.append(user)
    hp_m_content.append(content)
    
hp_mentions = pd.DataFrame({
    'User ID': hp_m_user,
    'Department': dept,
    'Department ID': dept_id,
    'Tweet content': hp_m_content
})

In [168]:
#International Development Department
r = requests.get('https://api.twitter.com/2/users/317018025/mentions?expansions=author_id&max_results=100', headers=headers)
ID_m = json.loads(r.text)['data']

dept_id = []; ID_m_user = []; ID_m_content = []; dept = []
for i in range(0, len(ID_m)):
    dept_id.append('317018025')
    dept.append('International Development')
    user = ID_m[i]['author_id']
    content = ID_m[i]['text']
    ID_m_user.append(user)
    ID_m_content.append(content)
    
ID_mentions = pd.DataFrame({
    'User ID': ID_m_user,
    'Department': dept,
    'Department ID': dept_id,
    'Tweet content': ID_m_content
})

In [169]:
#International History department
r = requests.get('https://api.twitter.com/2/users/253471591/mentions?expansions=author_id&max_results=100', headers=headers)
ih_m = json.loads(r.text)['data']

dept_id = []; ih_m_user = []; ih_m_content = []; dept = []
for i in range(0, len(ih_m)):
    dept_id.append('253471591')
    dept.append('International History')
    user = ih_m[i]['author_id']
    content = ih_m[i]['text']
    ih_m_user.append(user)
    ih_m_content.append(content)
    
ih_mentions = pd.DataFrame({
    'User ID': ih_m_user,
    'Department': dept,
    'Department ID': dept_id,
    'Tweet content': ih_m_content
})

In [170]:
#International Relations department
r = requests.get('https://api.twitter.com/2/users/237225532/mentions?expansions=author_id&max_results=100', headers=headers)
ir_m = json.loads(r.text)['data']

dept_id = []; ir_m_user = []; ir_m_content = []; dept = []
for i in range(0, len(ir_m)):
    dept_id.append('237225532')
    dept.append('International Relations')
    user = ir_m[i]['author_id']
    content = ir_m[i]['text']
    ir_m_user.append(user)
    ir_m_content.append(content)
    
ir_mentions = pd.DataFrame({
    'User ID': ir_m_user,
    'Department': dept,
    'Department ID': dept_id,
    'Tweet content': ir_m_content
})

In [171]:
#Language Centre Department
r = requests.get('https://api.twitter.com/2/users/179888345/mentions?expansions=author_id&max_results=100', headers=headers)
lang_m = json.loads(r.text)['data']

dept_id = []; lang_m_user = []; lang_m_content = []; dept = []
for i in range(0, len(lang_m)):
    dept_id.append('179888345')
    dept.append('Language Centre')
    user = lang_m[i]['author_id']
    content = lang_m[i]['text']
    lang_m_user.append(user)
    lang_m_content.append(content)
    
lang_mentions = pd.DataFrame({
    'User ID': lang_m_user,
    'Department': dept,
    'Department ID': dept_id,
    'Tweet content': lang_m_content
})

In [172]:
#Law Department
r = requests.get('https://api.twitter.com/2/users/532172035/mentions?expansions=author_id&max_results=100', headers=headers)
law_m = json.loads(r.text)['data']

dept_id = []; law_m_user = []; law_m_content = []; dept = []
for i in range(0, len(law_m)):
    dept_id.append('532172035')
    dept.append('LSE Law School')
    user = law_m[i]['author_id']
    content = law_m[i]['text']
    law_m_user.append(user)
    law_m_content.append(content)
    
law_mentions = pd.DataFrame({
    'User ID': law_m_user,
    'Department': dept,
    'Department ID': dept_id,
    'Tweet content': law_m_content
})

In [173]:
#Management Department
r = requests.get('https://api.twitter.com/2/users/26465977/mentions?expansions=author_id&max_results=100', headers=headers)
man_m = json.loads(r.text)['data']

dept_id = []; man_m_user = []; man_m_content = []; dept = []
for i in range(0, len(man_m)):
    dept_id.append('26465977')
    dept.append('Management')
    user = man_m[i]['author_id']
    content = man_m[i]['text']
    man_m_user.append(user)
    man_m_content.append(content)
    
man_mentions = pd.DataFrame({
    'User ID': man_m_user,
    'Department': dept,
    'Department ID': dept_id,
    'Tweet content': man_m_content
})

In [174]:
#Mathematics Department
r = requests.get('https://api.twitter.com/2/users/3044880371/mentions?expansions=author_id&max_results=100', headers=headers)
math_m = json.loads(r.text)['data']

dept_id = []; math_m_user = []; math_m_content = []; dept = []
for i in range(0, len(math_m)):
    dept_id.append('3044880371')
    dept.append('Mathematics')
    user = math_m[i]['author_id']
    content = math_m[i]['text']
    math_m_user.append(user)
    math_m_content.append(content)
    
math_mentions = pd.DataFrame({
    'User ID': math_m_user,
    'Department': dept,
    'Department ID': dept_id,
    'Tweet content': math_m_content
})

In [175]:
#Media and Communications Department
r = requests.get('https://api.twitter.com/2/users/207534677/mentions?expansions=author_id&max_results=100', headers=headers)
mc_m = json.loads(r.text)['data']

dept_id = []; mc_m_user = []; mc_m_content = []; dept = []
for i in range(0, len(mc_m)):
    dept_id.append('207534677')
    dept.append('Media and Communications')
    user = mc_m[i]['author_id']
    content = mc_m[i]['text']
    mc_m_user.append(user)
    mc_m_content.append(content)
    
mc_mentions = pd.DataFrame({
    'User ID': mc_m_user,
    'Department': dept,
    'Department ID': dept_id,
    'Tweet content': mc_m_content
})

In [176]:
#Methodology Department
r = requests.get('https://api.twitter.com/2/users/86921024/mentions?expansions=author_id&max_results=100', headers=headers)
met_m = json.loads(r.text)['data']

dept_id = []; met_m_user = []; met_m_content = []; dept = []
for i in range(0, len(met_m)):
    dept_id.append('86921024')
    dept.append('Methodology')
    user = met_m[i]['author_id']
    content = met_m[i]['text']
    met_m_user.append(user)
    met_m_content.append(content)
    
met_mentions = pd.DataFrame({
    'User ID': met_m_user,
    'Department': dept,
    'Department ID': dept_id,
    'Tweet content': met_m_content
})

In [177]:
#Philosophy, Logic and Scientific Method Department
r = requests.get('https://api.twitter.com/2/users/904251031/mentions?expansions=author_id&max_results=100', headers=headers)
phil_m = json.loads(r.text)['data']

dept_id = []; phil_m_user = []; phil_m_content = []; dept = []
for i in range(0, len(phil_m)):
    dept_id.append('904251031')
    dept.append('Philosophy, Logic and Scientific Method')
    user = phil_m[i]['author_id']
    content = phil_m[i]['text']
    phil_m_user.append(user)
    phil_m_content.append(content)
    
phil_mentions = pd.DataFrame({
    'User ID': phil_m_user,
    'Department': dept,
    'Department ID': dept_id,
    'Tweet content': phil_m_content
})

In [178]:
#Psychological and Behavioural Science Department
r = requests.get('https://api.twitter.com/2/users/1965000560/mentions?expansions=author_id&max_results=100', headers=headers)
psych_m = json.loads(r.text)['data']

dept_id = []; psych_m_user = []; psych_m_content = []; dept = []
for i in range(0, len(psych_m)):
    dept_id.append('1965000560')
    dept.append('Psychological and Behavioural Science')
    user = psych_m[i]['author_id']
    content = psych_m[i]['text']
    psych_m_user.append(user)
    psych_m_content.append(content)
    
psych_mentions = pd.DataFrame({
    'User ID': psych_m_user,
    'Department': dept,
    'Department ID': dept_id,
    'Tweet content': psych_m_content
})

In [179]:
#Social Policy Department
r = requests.get('https://api.twitter.com/2/users/2472172578/mentions?expansions=author_id&max_results=100', headers=headers)
sp_m = json.loads(r.text)['data']

dept_id = []; sp_m_user = []; sp_m_content = []; dept = []
for i in range(0, len(sp_m)):
    dept_id.append('2472172578')
    dept.append('Social Policy')
    user = sp_m[i]['author_id']
    content = sp_m[i]['text']
    sp_m_user.append(user)
    sp_m_content.append(content)
    
sp_mentions = pd.DataFrame({
    'User ID': sp_m_user,
    'Department': dept,
    'Department ID': dept_id,
    'Tweet content': sp_m_content
})

In [180]:
#Sociology Department
r = requests.get('https://api.twitter.com/2/users/1671486960/mentions?expansions=author_id&max_results=100', headers=headers)
socio_m = json.loads(r.text)['data']

dept_id = []; socio_m_user = []; socio_m_content = []; dept = []
for i in range(0, len(socio_m)):
    dept_id.append('1671486960')
    dept.append('Sociology')
    user = socio_m[i]['author_id']
    content = socio_m[i]['text']
    socio_m_user.append(user)
    socio_m_content.append(content)
    
socio_mentions = pd.DataFrame({
    'User ID': socio_m_user,
    'Department': dept,
    'Department ID': dept_id,
    'Tweet content': socio_m_content
})

In [181]:
#Statistics Department
r = requests.get('https://api.twitter.com/2/users/420282103/mentions?expansions=author_id&max_results=100', headers=headers)
stats_m = json.loads(r.text)['data']

dept_id = []; stats_m_user = []; stats_m_content = []; dept = []
for i in range(0, len(stats_m)):
    dept_id.append('420282103')
    dept.append('Statistics')
    user = stats_m[i]['author_id']
    content = stats_m[i]['text']
    stats_m_user.append(user)
    stats_m_content.append(content)
    
stats_mentions = pd.DataFrame({
    'User ID': stats_m_user,
    'Department': dept,
    'Department ID': dept_id,
    'Tweet content': stats_m_content
})

In [230]:
mentions = pd.concat([acc_mentions, anth_mentions, econ_mentions, econhist_mentions,
         finance_mentions, gender_mentions, geo_mentions, gov_mentions,
         hp_mentions, ID_mentions, ih_mentions, ir_mentions, lang_mentions,
         law_mentions, man_mentions, math_mentions, mc_mentions, met_mentions,
         phil_mentions, psych_mentions, sp_mentions, socio_mentions, stats_mentions], ignore_index=True)

mentions

,User ID,Department,Department ID,Tweet content
0,1898598396,Accounting,4900666161,A great opportunity for a range of #research p...
1,1456252200825597963,Accounting,4900666161,Develop a financial and managerial accounting ...
2,21861323,Accounting,4900666161,Join our MRes/PhD in Accounting information se...
3,21861323,Accounting,4900666161,Join our MRes/PhD in Accounting information se...
4,1332217117844905985,Accounting,4900666161,Erasmus Accounting Workshop!\n\nSpeaker: \n- C...
...,...,...,...,...
2295,768457181846962176,Statistics,420282103,⌛️ Not long to wait until the @LSEStatistics n...
2296,768457181846962176,Statistics,420282103,@julioadl @LSEnews @ImperialBiz @LSEStatistics...
2297,21643972,Statistics,420282103,"VIDEO: Drawing from their recent book, Data Fe..."
2298,1254677167,Statistics,420282103,@LSEStatistics @kanarinka @laurenfklein Very i...


To determine the sentiment of the mentions, I'll be using TextBlob. Two useful things in TextBlob are subjectivity and polarity. Subjectivity tells us what kind of mention this tweet is. For example, the more subjective a tweet is, the more opinionated or emotion heavy it is. Whereas, a more objective tweet contains more facts. The more important classification score that I'll be making use of is polarity. Polarity operates on a scale of [-1, 1] where -1 represents a negative tweet, and 1 represents a positive tweet.

In [231]:
#Sentiment analysis code using TextBlob
from textblob import TextBlob

def Subjectivity(tweet):
    return TextBlob(tweet).sentiment.subjectivity
    
#Polarity
def Polarity(tweet):
    return TextBlob(tweet).sentiment.polarity

In [232]:
mentions['Subjectivity'] = mentions['Tweet content'].apply(Subjectivity)
mentions['Polarity'] = mentions['Tweet content'].apply(Polarity)

In [233]:
def sentiment_analysis(score):
    if score < 0:
        return 'Negative'
    elif score == 0 :
        return 'Neutral'
    elif score > 0:
        return 'Positive'
    
mentions['Sentiment'] = mentions['Polarity'].apply(sentiment_analysis)

In [234]:
mentions

,User ID,Department,Department ID,Tweet content,Subjectivity,Polarity,Sentiment
0,1898598396,Accounting,4900666161,A great opportunity for a range of #research p...,0.375000,0.400000,Positive
1,1456252200825597963,Accounting,4900666161,Develop a financial and managerial accounting ...,0.000000,0.000000,Neutral
2,21861323,Accounting,4900666161,Join our MRes/PhD in Accounting information se...,0.400000,0.000000,Neutral
3,21861323,Accounting,4900666161,Join our MRes/PhD in Accounting information se...,0.400000,0.000000,Neutral
4,1332217117844905985,Accounting,4900666161,Erasmus Accounting Workshop!\n\nSpeaker: \n- C...,0.500000,0.250000,Positive
...,...,...,...,...,...,...,...
2295,768457181846962176,Statistics,420282103,⌛️ Not long to wait until the @LSEStatistics n...,0.233333,0.015625,Positive
2296,768457181846962176,Statistics,420282103,@julioadl @LSEnews @ImperialBiz @LSEStatistics...,0.351515,0.312121,Positive
2297,21643972,Statistics,420282103,"VIDEO: Drawing from their recent book, Data Fe...",0.525000,0.000000,Neutral
2298,1254677167,Statistics,420282103,@LSEStatistics @kanarinka @laurenfklein Very i...,0.300000,0.200000,Positive


In [235]:
def positive(text):
    if text == 'Positive':
        return 1
    else:
        return 0
    
def neutral(text):
    if text == 'Neutral':
        return 1
    else:
        return 0
    
def negative(text):
    if text == 'Negative':
        return 1
    else:
        return 0
    
mentions['Positive'] = mentions['Sentiment'].apply(positive)
mentions['Neutral'] = mentions['Sentiment'].apply(neutral)
mentions['Negative'] = mentions['Sentiment'].apply(negative)

In [236]:
mentions_sum = mentions.groupby(['Department'], as_index=False).sum()
mentions_sum

,Department,Subjectivity,Polarity,Positive,Neutral,Negative
0,Accounting,24.917578,10.363927,35,57,8
1,Anthropology,38.961540,18.907667,54,38,8
2,Economic History,25.081843,12.858353,45,47,8
3,Economics,29.789894,9.742460,50,35,15
4,Finance,32.508878,19.157927,49,45,6
5,Gender Studies,36.155846,20.620066,56,38,6
6,Geography and Evironment,29.301803,13.161900,53,42,5
7,Government,23.576111,22.024097,56,44,0
8,Health Policy,25.505852,22.079053,41,58,1
9,International Development,34.056413,12.649089,42,40,18


In [237]:
dept_eng["Mentions' subjectivity"] = mentions_sum['Subjectivity']
dept_eng["Mentions' polarity"] = mentions_sum['Polarity']
dept_eng['Positive mentions'] = mentions_sum['Positive']
dept_eng['Neutral mentions'] = mentions_sum['Neutral']
dept_eng['Negative mentions'] =  mentions_sum['Negative']
dept_eng

,Department,Retweet count,Reply count,Like count,Quote count,Follower count,Total engagement,Engagement ratio,Mentions' subjectivity,Mentions' polarity,Positive mentions,Neutral mentions,Negative mentions
0,Accounting,288,1,94,3,2513,386,0.153601,24.917578,10.363927,35,57,8
1,Anthropology,673,4,388,16,6642,1081,0.162752,38.961540,18.907667,54,38,8
2,Economics,368,6,283,8,35620,665,0.018669,25.081843,12.858353,45,47,8
3,Economics History,144,66,346,13,3874,569,0.146877,29.789894,9.742460,50,35,15
4,Finance,391,1,38,3,2478,433,0.174738,32.508878,19.157927,49,45,6
5,Gender Studies,1592,10,308,19,19644,1929,0.098198,36.155846,20.620066,56,38,6
6,Geography and Environment,471,4,267,13,12553,755,0.060145,29.301803,13.161900,53,42,5
7,Government,377,7,131,7,24726,522,0.021111,23.576111,22.024097,56,44,0
8,Health Policy,330,22,415,13,7635,780,0.102161,25.505852,22.079053,41,58,1
9,International Development,290,5,186,15,12130,496,0.040890,34.056413,12.649089,42,40,18


### Part 2: Data Visualisation

Now that we've found how many mentions are generally positive, neutral, and negative, the dataset is complete. In the data analysis after this, I'll be using both the engagement dataset and also the dataset of tweet mentions. Here are the list of important datasets for easier tracking:
* dept_eng: Represents an aggregate of each department's engagement statistics
* mentions: Contains the last 100 mentions of each department
* tweets_stats: Contains engagement information of 100 the last 100 tweets of each department

On observation, these are some graphs that I could draw to decide whether Twitter is an effective social media platform:
* Departments and their retweet count 
* Departments and their like count 
* Retweet engagements vs normal tweet engagements 
* Overall following count vs follower to engagement ratios 
* Total comments and ratio of comment sentiments
* Network graph of the users who have mentioned the department's accounts